# Kastroulis, Angelo

# Neural Networks for SQL Query Optimization

## The problem

The open research question pursued in this project is whether Neural Networks have a place in the optimizer of SQL database systems. Optimizers have to make decisions as to whether to probe an index, scan the table, or add more workers (concurrency) among other things. These tend to be complex decisions, and are traditionally based on the optimizer's view of the cost of the operations and statistics to predict what the selectivity of the query will be. While the optimizer can't be directly manipulated, it can be influenced through knobs (tuning parameters). Even so, the optimizer rarely will choose the most optimal query execution path.

## Data Source

The TPC Benchmark™H (TPC-H) is a decision support benchmark. It consists of a suite of business oriented ad-hoc queries and concurrent data modifications. I used the TPC-H Lineitem table and 16 variations of Query 6 to give me a range of selectivity.

Those variations of Query 6 on Lineitem table were run on 3 AWS EC2 instances on Postgres 10 (a memory optimized R5D.4xlarge, a compute optimized C5D.4xlarge, and a general M5D.2xlarge). Lineitem was generated using TPCH tools with a scale factor of 10, giving me ~60M rows. I then ran 5 test runs of all permutations of queries and system attributes and collected the data. The compiled dataset is called performance_data.csv (7.9MB) and performance_data_one-hot.csv (9.9 MB).

## General Approach

While data generation is out of scope for this report, I am including those files in case someone wants to do that (and the discussion in those notebooks is also interesting). It took days to generate on each server. The combined set (performance_data) is included for your use. Here are the steps to generate data:

- Generate TPC-H lineitem table with scale factor 10 (instructions in _BACKGROUND.ipynb_ ~60GB data)
- Run the SQL scripts on the server (_TEST_\__PACKAGE_) ~48-72 hours per server).
- Combine the raw logs (_ExplainCleanup.ipynb_)
- Clean and combine to final set (_FeatureEngineering.ipynb_).

Modeling approach:

- Normalize the data using pandas
- Create a NN with 7 layers. (Dense 128 RELU, Dense 128 Linear, Dense 64 RELU, Dense 64 Linear, Dense 128 RELU, Dense 128 Linear, Dense final prediction layer with 1 neuron). For a discussion on why this architecture, and what others were tested, see ModelMLP.ipynb)
- Train and compare to what the optimizer thought was best (the optimizer was separated to separate files named XXX_control.txt, where XXX is the server).

## Required Software

- Postgres (if you want to re-generate data)
- Python 3
- Jupyter
- Pandas, matplotlib, numpy
- Tensorflow & Keras

## Interesing Files
- ``data/performance_data.csv`` Final, combined and cleaned data file
- ``data/performance_data_one_hot.csv`` Data file with one-hot encoded categorical variables
- ``data/performance_data_one_hot_augmented.csv`` Above file with generated evenly distributed classes (not used)
- ``FinalModel.h5`` The saved final model
- ``Background.ipynb`` A deeper discussion on TPCH, the systems, and the queries and instructions on how to install and generate your own data.
- ``FeatureEngineering.ipynb`` A deeper discussion of the data and its transformation
- ``Model-MLP.ipynb`` A deeper discussion on the various model architectures attempted
- ``TEST_PACKAGE`` folder. The SQL Scripts (if you want to generate data).
- ``TEST_PACKAGE/ExplainCleanup.ipynb`` Used to take the raw logs and create a server-specific data file. This takes the raw results from the SQL queries and produces an output that is loaded by the FeatureEngineering notebook to produce the performance_data.csv file.

## Outcome

The NN approach outperformed Postgres 10's optimizer in nearly all cases in terms of path prediction accuracy. The NN also did remarkably well detecting better paths on the different systems, whereas the optimizer did not.

## Youtube Videos
- **15 min**: https://www.youtube.com/watch?v=1-Upc0vLqC0
- **2 min**: https://www.youtube.com/watch?v=JDZ6P9bh7n0